In [1]:
import pandas as pd
import numpy as np
import catboost as cat

In [2]:
train = pd.read_csv('./Geochemistry Data/CNS_gas_train.csv')

In [3]:
train[:1]

,Country,WELL_NAME,OGA Well Name,WH_SITETYPE,WH_FIELD,WH_BLOCK,WH_LAT,WH_LONG,WH_COORD_SYS,WH_OPERATOR,...,DD_G_H2,DN_G_N2,DD_G_GAS,DD_G_NC1,DD_G_NC2,DD_G_NC3,DD_G_IC4,DD_G_NC4,DD_G_IC5,DD_G_NC5
0,NaN,Well name,Well name,Site type,Field name,Block name,Latitude,Longitude,Location CRS,Operator,...,dDeuterium on H2,dN15 N2,dDeuterium total GAS,dDeuterium nC1,dDeuterium nC2,dDeuterium nC3,dDeuterium iC4,dDeuterium nC4,dDeuterium iC5,dDeuterium nC5


In [4]:
train.drop([0], inplace=True)

In [5]:
test = pd.read_csv('./Geochemistry Data/CNS_gas_test.csv')

In [6]:
test_colls = 'GAS_C1, GAS_C2, GAS_C3, GAS_IC4, GAS_NC4, GAS_IC5, GAS_NC5'.replace(',', '').split()

In [7]:
coll_mask = (train.describe().loc[['unique']].values > 2)[0]

In [8]:
coll_mask

array([False,  True,  True, False,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True, False,  True,  True,  True, False, False, False,
       False, False,  True, False,  True,  True, False,  True, False,
       False, False,  True,  True,  True, False,  True,  True,  True,
        True,  True,  True,  True,  True, False,  True,  True,  True,
        True, False,  True,  True,  True,  True, False,  True,  True,
        True, False, False,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True, False,
        True,  True, False,  True, False, False, False, False, False,
       False])

In [9]:
good_colls = np.array(train.columns)[coll_mask]

In [10]:
train.fillna(0, inplace=True)
test.fillna(0, inplace=True)

In [11]:
print("GOOD COLS: ", len(good_colls))

GOOD COLS:  64


In [12]:
train_colls = list(set(good_colls) - set(test_colls))

In [13]:
X = train[train_colls]
y = train[test_colls]

In [14]:
X

,SAMPLE_ID,CSIA_NC1,WELL_NAME,CSIA_NC3,D_G_ACQ_DATE,WH_LONG,SH_SAMPLE_SUB_TYPE,SH_ORIG,GAS_O2,WH_DR_ELEV_M,...,CSIA_CONC_C1,WH_SPUD_DATE,SAMPLE_ID_GDB,WH_SPUD_YEAR,GAS_CO2,GAS_HE,D_G_METHOD,SH_COMMENT,CSIA_NC2,SH_ORIG_DESC
1,EXPRO_41219,-42.1,023/22A-A12 (A12),-28.2,23-05-16 0:00,0,Subsurface,SEPR,0,0,...,0,0,0,0,1.8,0,CSIA,1379-C1-F,-28.8,0
2,EXPRO_41220,-42.1,023/22A-A12 (A12),-28.2,23-05-16 0:00,0,Subsurface,SEPR,0,0,...,0,0,0,0,1.8,0,CSIA,A2442,-28.8,0
3,EXPRO_41223,-70.8,023/22A-A12 (A12),-26.1,24-05-16 0:00,0,Subsurface,ISOT,0,0,...,1.7,0,0,0,0.037,0,CSIA,Gas% 0.5,-35.5,0
4,EXPRO_41224,-69.8,023/22A-A12 (A12),-28.3,24-05-16 0:00,0,Subsurface,ISOT,0,0,...,0.97,0,0,0,0.033,0,CSIA,Gas% 1.15,-33.1,0
5,EXPRO_41225,-67.7,023/22A-A12 (A12),-29.2,24-05-16 0:00,0,Subsurface,ISOT,0,0,...,0.72,0,0,0,0.036,0,CSIA,Gas% 1.24,-32.9,0
6,EXPRO_41226,-62.8,023/22A-A12 (A12),-32.3,24-05-16 0:00,0,Subsurface,ISOT,0,0,...,0.69,0,0,0,0.033,0,CSIA,Gas% 0.8,-33.1,0
7,EXPRO_41227,-54.9,023/22A-A12 (A12),-31.5,24-05-16 0:00,0,Subsurface,ISOT,0,0,...,0.79,0,0,0,0.027,0,CSIA,Gas% 0.97,-31.2,0
8,EXPRO_41228,-52.3,023/22A-A12 (A12),-30.4,24-05-16 0:00,0,Subsurface,ISOT,0,0,...,0.24,0,0,0,0.027,0,CSIA,Gas% 0.4,-30,0
9,EXPRO_41229,-48.6,023/22A-A12 (A12),-30.2,24-05-16 0:00,0,Subsurface,ISOT,0,0,...,0.19,0,0,0,0.043,0,CSIA,Gas% 0.32,-29.6,0
10,EXPRO_41230,-43.8,023/22A-A12 (A12),-29.1,24-05-16 0:00,0,Subsurface,ISOT,0,0,...,0.13,0,0,0,0.034,0,CSIA,Gas% 0.2,-29,0


In [15]:
regressor = cat.CatBoostRegressor()

In [22]:
regressor.fit(X.values, y.values[:,0])

CatBoostError: Bad value for num_feature[non_default_doc_idx=0,feature_idx=0]="EXPRO_41219": Cannot convert 'b'EXPRO_41219'' to float